In [ ]:
import bucket_manager.bucket_manager as bm

In [ ]:
import os
import json
s3_host = 'https://s3.echo.stfc.ac.uk/auth/1.0'
api = 'swift'
# keys = bm.get_keys(api)
with open(os.path.expanduser('~/lsst-swift-credentials.json'), 'r') as kf:
    keys = json.load(kf)
user = keys['user']
secret_key = keys['secret_key']


In [ ]:
s3 = bm.get_conn_swift(user, secret_key, s3_host)

In [ ]:
bucket_list = bm.bucket_list_swift(s3)

In [ ]:
bucket_list

In [ ]:
bucket_name = 'LSST-IR-FUSION-Butlers'

In [ ]:
current_objects = bm.object_list_swift(s3, bucket_name, full_listing=False)

In [ ]:
current_objects

In [ ]:
len(current_objects)

In [ ]:
current_contents = bm.get_bucket_contents_swift(s3, bucket_name)

In [ ]:
import pandas as pd
current_contents = pd.DataFrame.from_dict(current_contents)
current_contents

In [ ]:
for key in current_objects:
    if type(key) == str:
        existing_zip_contents = None
        if key.endswith('.zip'):
            print('.', end='', flush=True)
            try:
                existing_zip_contents = str(s3.get_object(bucket_name,''.join([key,'.metadata']))[1].decode('UTF-8')).split('|') # use | as separator
            except Exception as e:
                try:
                    existing_zip_contents = s3.head_object(bucket_name,key)['x-object-meta-zip-contents'].split('|') # use | as separator
                    if len(existing_zip_contents) == 1:
                        existing_zip_contents = existing_zip_contents[0].split(',') # revert to comma if no | found
                except KeyError as e:
                    print('KeyError: ', e)
                except Exception as e:
                    print('Exception: ', e)
            if existing_zip_contents:
                print(existing_zip_contents)
        else:
            print('Not a zip file')
    else:
        print('Not a string')

Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip file
Not a zip 

KeyboardInterrupt: 